# PACE Chlorophyll A data over a region in the NorthAtlantic Ocean

This notebook demonstrates access and subsetting to PACE Ocean Color Data. Broad information about the dataset can be found on the PACE website (see [here](https://oceandata.sci.gsfc.nasa.gov))

**Requirements to run this notebook**
1. Have an Earth Data Login account
2. Have a Bearer Token.
3. Knowledge of a collection concept ID (or DOI) of a dataset.
4. Internet connection.


**Objectives**
 
Use [pydap](https://pydap.github.io/pydap/) to demonstrate

- Discovery of OPeNDAP granules from a collection of interest, further filtering with a time range.
- Use of `tokens` to authenticate (from earthaccess).
- Accessing Level 3 `PACE`, and identify an area of interest.
- Subset and enable the `Constraint Expression` to make its way to the OPeNDAP server.
- Store the subset locally.

`Author`: Miguel Jimenez-Urias, '25

In [ ]:
from pydap.client import get_cmr_urls, consolidate_metadata, open_url
from pydap.net import create_session
import xarray as xr
import earthaccess
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

### Use a Session as a Metadata Database and Authentication

Pydap can make use of a `requests_cache.CachedSession` object, which can be used as

- Authentication (via netrc or token).
- Database Management (sqlite3 as backend). Only caches necessary data.
- Stream data.

Below we initialize the session, inheriting auth credentials from earthaccess, and point to local database `PACE.sqlite` store in the `/data` folder



In [ ]:
auth = earthaccess.login(strategy="interactive", persist=True)
session = create_session(use_cache=True, session=earthaccess.get_requests_https_session(), cache_kwargs={'cache_name': 'data/PACE'})
session.cache.clear()

### Query CMR for OPeNDAP URLs

We query NASA's CMR for all opendap urls associated with 4km, daily, Level 3 Gridded Chlorophyll A, version 3.1. For this, you can use Earthdata Search to identify the Concept Collection ID. We focus for 2 months of data this year.





In [ ]:
PACE_ccid = "C3620140255-OB_CLOUD" # version 3.1
time_range=[dt.datetime(2025, 6, 1), dt.datetime(2025, 8, 1)]
urls = get_cmr_urls(ccid=PACE_ccid, limit=1000, time_range=time_range) # limit by default = 50



```{note}
This collection had two version of the same data. We are interested in the `4km` resolution. One way to be certain that any number of opendap urls belong to the same collection, is by inspecting the url strings (unnecessary in most cases)
```


Further, we subset by variable name, retaining only `lat`, `lon`, and `chlor_a` variables. These are the variables of interest, and discarding variables reduces the amount of time it takes for `Xarray` to process the metadata


```{note}
Xarray has a .drop_vars method, but these variables are dropped AFTER creating the dataset. If a dataset has O(1000) variables, `Xarray` would process the variables first, and then drop them. With OPeNDAP, you can instruct the server a priori which variables Xarray needs to process.

```


In [ ]:
CEs = "?dap4.ce=/lat;/lon;/chlor_a"

In [ ]:
_urls = [url for url in urls if '4km' in url and "DAY" in url]
pace_urls = [url.replace("https", "dap4") + CEs for url in _urls] # a dap4 schema implies a DAP4 protocol in pydap.
len(pace_urls)

In [ ]:
pace_urls[:4]

### Consolidate metadata aggregation

In here, the dimensions and some metadata is eagerly download. This step is not necessary but provides a major boost in the performance when using pydap  as a backend of xarray. 

The metadata is stored as sqlite, and can persist through sessions.

```{note}
This collection only defines the time dimension (or coordinate) in the metadata. Because there is no  variable in the remote file associated with the time coordinate, there is no need to define a concatenating dimension (`concat_dim`) when running the code below. Almost always there is a `concat_dim`, but not in this special case. 
```


In [ ]:
%%time
consolidate_metadata(pace_urls, session=session)

### Create a single Dataset object

We use `Xarray` with `pydap` as the backend. `Pydap` is a backend that is always installed when installing `Xarray`, but not yet the default when working with opendap urls. As a result, we must define it.


In [ ]:
%%time
ds = xr.open_mfdataset(
    pace_urls, 
    engine='pydap', 
    session=session, 
    parallel=True, 
    concat_dim='time',  # <------ a time dimension will be created 
    combine='nested',
)

In [ ]:
ds


The dataset above represent a metadata representation of the entire data of interest. It is chunked, and only the dimensions lat and lon have been downloaded.

```{note}
A named dimension `time` was created, along with all remote granules were concatenated.
```

In [ ]:
print('Uncompressed dataset size [GBs]: ', ds.nbytes / 1e9)

### Data-proximate subsetting: How to with OPeNDAP, Pydap, and Xarray?


While it is possible to download the entire dataset, it is better practice to subset it first! There are many tools to subset data in a data-proximate way, and OPeNDAP is one of them. We need to identify the subset first and pass it to the server.

Identifying the subset enables users to potentially construct Constraint Expressions, which are used to instruct the OPeNDAP server the subset of interest. But with Pydap and Xarray, these are tools that interactively construct slices and can request subsets, hiding the abstraction. Below we demonstrate how to pass the subset to the OPeNDAP server so that Xarray does less work.

### Identify spatial subset

In this case, we are interested in a spatial subset. The data is Level 3 data (gridded) so latitude and longitude are uniform. Moreover, these are 1D, and have already been downloaded into memory!

In [ ]:
lat, lon = ds['lat'].values, ds['lon'].values

### Say we define the area of interest

In [ ]:
# Min/max of lon values
minLon, maxLon = -96, 10

# Min/Max of lat values
minLat, maxLat = 6, 70

In [ ]:
iLon = np.where((lon>minLon)&(lon < maxLon))[0]
iLat= np.where((lat>minLat)&(lat < maxLat))[0]

So the slices are simply the first and last elements of iLon and iLat!!:

```python
lon_slice = slice(iLon[0], iLon[-1])
lat_slice = slice(iLat[0], iLat[-1])
```


```{warning}
Not all array values of `lat/lon` coordinates are monotonic. Always make sure that is the case, even when data is Level 3 or Level 4

```

### Download only the subset


With `Xarray`, subsetting syntax is similar to pandas. For example
```
ds.isel(lon=lon_slice, lat_slice)
```

Produces a client-side subset. However

```{warning}
When using opendap, slice as above and then downloading data will only subset by variable, downloading all of the data in most cases, only to then be further subset by xarray.  
```



To ensure the slices are sent to the remote server, we must `chunk the dataset` when creating it. This will guarantee that the subset will be mostly done on the server side. You `MUST` choose the chunk size to match the size of the slice, as shown below.

In [ ]:
%%time
ds = xr.open_mfdataset(
    pace_urls, 
    engine='pydap', 
    session=session, 
    parallel=True, 
    concat_dim='time',  # <------ a time dimension will be created 
    combine='nested',
    chunks = {'lon': len(iLon), 'lat':len(iLat)} #  <----------- This instructs the OPeNDAP server to subset in space
)
ds

In [ ]:
ds["chlor_a"] ## inspect the chunk of the data

We now ensure `Xarray` will treat this subset as an individual chunk (per variable)

In [ ]:
ds = ds.isel(lon=slice(iLon[0], iLon[-1]+1), lat=slice(iLat[0], iLat[-1]+1)).chunk({'lon': len(iLon), 'lat':len(iLat)})

### Finally

Store data locally, as `NetCDF4`. At this stage:

- Data is downloaded (spatially subsetted via `OPeNDAP`)
- Data is resampled (via `Xarray`)

In [ ]:
%%time
ds.to_netcdf("data/pace_subset.nc4", mode='w')

### We inspect our data


In [ ]:
mds = xr.open_dataset("data/pace_subset.nc4")
mds

In [ ]:
print("Size of downloaded subset: ", mds.nbytes/1e9)